In [1]:
import pandas as pd
import numpy as np

In [2]:
# Leer archivos finales de supermercados
plazavea = pd.read_parquet('../supermercados/plazavea/plazavea.parquet')
metro = pd.read_parquet('../supermercados/metro/metro.parquet')
wong = pd.read_parquet('../supermercados/wong/wong.parquet')
vivanda = pd.read_parquet('../supermercados/vivanda/vivanda.parquet')

In [ ]:
# Provisional
# df = vivanda
# df.sample(n=3)

In [3]:
# Concatenar todos los dataframes en df
df = pd.concat([plazavea, metro, wong, vivanda], ignore_index=True)

In [8]:
df.sample(n=10)

,product_name,link,descripcion_producto,price,source,categories
13205,Licuadora Oster® ActiveSense con Blend-N-Go B...,https://www.plazavea.com.pe/licuadora-oster-ac...,plazaVea,975.00,plazaVea,"[Electrohogar, Electrodomésticos de Cocina, Li..."
105822,Agenda Arti Creativo Oslo Formato B5 Negro,https://www.wong.pe/agenda-arti-creativo-oslo-...,Wong,31.00,Wong,"[Libros y Librería, Agendas 2024]"
6420,Comida para Gatos Adultos Esterilizados Braver...,https://www.plazavea.com.pe/comida-para-gatos-...,plazaVea,269.90,plazaVea,"[Mascotas, Comida para gatos, Croquetas y comi..."
108838,Queso Maduro Cheddar Rallado Crystal Farms 226g,https://www.wong.pe/queso-maduro-cheddar-ralla...,Wong,26.80,Wong,"[Lácteos, La Quesería, Quesos Semiblandos]"
76900,Ilko Sartén Antiadherente 28 cm Silver Industrial,https://www.metro.pe/ilko-sarten-antiadherente...,Metro,89.99,Metro,"[Hogar y Bazar, Cocina, Sartenes y Woks]"
3757,Escoba para el Hogar Resistente de Verde Y+Pap...,https://www.plazavea.com.pe/escoba-para-el-hog...,plazaVea,53.90,plazaVea,"[Limpieza, Accesorios de Limpieza, Escobas y R..."
8242,Apoquel Tratamiento de la Dermatitis Alergica ...,https://www.plazavea.com.pe/apoquel-tratamient...,plazaVea,117.80,plazaVea,"[Mascotas, Salud e higiene para perros, Más pr..."
55880,Botines Azaleia LIDE-658 Negro,https://www.plazavea.com.pe/botines-azaleia-li...,plazaVea,0.00,plazaVea,"[Zapatos, Zapatos Mujer, Botas y Botines Mujer]"
98676,Molinillo La Hacienda Sal Pimienta,https://www.wong.pe/molinillo-la-hacienda-sal-...,Wong,64.39,Wong,"[Hogar y Bazar, Menaje de Mesa, Accesorios de ..."
81420,Pizza Hawaiana Wong Familiar (30 cm),https://www.metro.pe/pizza-hawaiana-wong-famil...,Metro,21.50,Metro,"[Comidas y Rostizados, Comidas, Pizzas]"


In [9]:
df.shape

(126551, 6)

In [10]:
df.to_parquet('productos_supermercados.parquet')

In [ ]:
df['categories'] = df['categories'].apply(
    lambda x: ', '.join(x) if isinstance(x, (list, np.ndarray)) else str(x)
).fillna("Sin categoría")

# Utilizamos ChromaDB y Embeddings

In [ ]:
!pip install chromadb sentence-transformers --quiet

In [ ]:
!pip install --upgrade jupyter ipywidgets --quiet

In [ ]:
!pip install tqdm --quiet

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
from chromadb.config import Settings
from chromadb import Client
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm, trange

In [ ]:
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer

# Configuración para guardar ChromaDB de forma permanente
client = PersistentClient(path="./chroma_db")  # Carpeta donde se guardará la base de datos

# Crear una colección
collection = client.get_or_create_collection(name="product_embeddings")

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Inicializar variables para el contador
batch_size = 100
total_records = len(df)

# Iterar en bloques de 100 registros
for i in range(0, total_records, batch_size):
    batch = df.iloc[i:i+batch_size]  # Seleccionar un bloque de registros
    
    # Extraer y generar embeddings para product_name
    product_names = batch['product_name'].astype(str).tolist()
    product_name_embeddings = embedding_model.encode(product_names).tolist()
    
    # Extraer y generar embeddings para categories (listas convertidas a texto)
    categories = batch['categories'].apply(
        lambda x: ', '.join(x) if isinstance(x, (list, np.ndarray)) else str(x)
    ).tolist()
    categories_embeddings = embedding_model.encode(categories).tolist()
    
    # Crear metadatos para el bloque actual
    metadata = [
        {
            "source": row['source'],
            "link": row['link'],
            "price": row['price'],
            "categories": ', '.join(row['categories']) if isinstance(row['categories'], (list, np.ndarray)) else str(row['categories'])
        }
        for _, row in batch.iterrows()
    ]
    
    # Insertar datos del bloque en la colección
    collection.add(
        embeddings=categories_embeddings,  # Usar embeddings de categories para consultas primarias
        metadatas=metadata,
        documents=product_names,  # Guardar los nombres de los productos como documentos
        ids=[str(idx) for idx in range(i, min(i+batch_size, total_records))]
    )

    # Imprimir progreso
    # print(f"{min(i+batch_size, total_records)} de {total_records} registros procesados.")

print("Embeddings y metadatos guardados en ChromaDB de forma permanente.")


In [ ]:
# Listar todas las colecciones disponibles
collections = client.list_collections()
print([col.name for col in collections])

In [ ]:
# Cargar la colección específica
collection = client.get_collection(name="product_embeddings")

In [ ]:
# Obtener datos por ID
data_by_id = collection.get(ids=["0", "1", "2"])  # IDs específicas
print(data_by_id)

In [ ]:
def search_products(query, precision=0.5, top_k=10):
    """
    Realiza una búsqueda en la base de datos ChromaDB por categorías y filtra por nombres de productos.
    
    Args:
        query (str): Texto de consulta.
        precision (float): Umbral de similitud (0.0 a 1.0) para los resultados relevantes.
        top_k (int): Número máximo de resultados a devolver en la búsqueda inicial.

    Returns:
        list: Lista de resultados relevantes con ID, nombre del producto, metadatos, y distancia.
    """
    # Generar embedding para la consulta
    query_embedding = embedding_model.encode([query]).tolist()
    
    # Primera búsqueda: categorías
    category_results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=['documents', 'metadatas', 'distances']
    )
    
    # Filtrar resultados relevantes por precisión en categorías
    filtered_results = []
    for i, distance in enumerate(category_results['distances'][0]):
        if distance <= precision:  # Solo considerar resultados dentro del umbral de precisión
            filtered_results.append({
                "id": category_results['ids'][0][i],
                "product_name": category_results['documents'][0][i],
                "metadata": category_results['metadatas'][0][i],
                "distance": distance
            })
    
    # Si no hay resultados relevantes por categoría, retornar vacío
    if not filtered_results:
        return []
    
    # Segunda búsqueda: filtrar por nombre del producto dentro de los resultados de categorías
    refined_results = []
    for result in filtered_results:
        product_name = result["product_name"]
        product_name_embedding = embedding_model.encode([product_name]).tolist()
        similarity = embedding_model.similarity(query_embedding, product_name_embedding)
        if similarity >= precision:  # Filtrar por umbral de precisión
            result["similarity"] = similarity
            refined_results.append(result)

    return refined_results


In [ ]:
# Ejemplo de búsqueda
query = "aceite 5W30 lubricante"
precision = 0.8
top_k = 10

results = search_products(query, precision=precision, top_k=top_k)

# Mostrar resultados
for result in results:
    print(result)


In [ ]:
# # Crear un embedding para consulta
# query_text = "aceite para freir papas"
# query_embedding = embedding_model.encode([query_text]).tolist()
# 
# # Buscar los registros más cercanos
# results = collection.query(
#     query_embeddings=query_embedding,
#     n_results=5,  # Número de resultados más cercanos
#     include=['embeddings', 'metadatas', 'documents', 'distances']
# )
# print(results)
